In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

In [23]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

#testing purposes
#df_train0 = pd.read_csv("train.csv")
#df_train = df_train0.loc[df_train0.shape[0]/10*7:df_train0.shape[0]/10*8]
#df_test = df_train0.loc[df_train0.shape[0]/10:df_train0.shape[0]/2]

In [24]:
df_test.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
100000,c1cc2sc3cc(-c4cccc5nsnc45)c4=C[SiH2]C=c4c3c2s1,1,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.74
100001,[nH]1ccc2Cc3cc4cc(cnc4cc3-c12)-c1cccc2nsnc12,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.90
100002,[nH]1ccc2oc3c(ccc4cc(sc34)-c3cccc4ccccc34)c12,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,2.64
100003,c1sc(-c2ccc(o2)-c2sc(-c3scc4occc34)c3[se]ccc23...,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.74
100004,C1=Cc2c(C1)c1cnc3cc(ncc3c1c1nsnc21)-c1ccccc1,1,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,2.07


In [25]:
#df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
700000,c1sc(c2[SiH2]C=Cc12)-c1cc2c3cscc3c3c4cnccc4[se...,1,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,2.43
700001,c1cc2c3nsnc3c3cc4cc(-c5scc6[nH]ccc56)c5nsnc5c4...,0,0,0,0,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,1.60
700002,[nH]1c2cc(sc2c2c1c1c3ccccc3ccc1c1=CCC=c21)-c1c...,1,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,2.01
700003,c1cc2oc3c(ccc4cc(-c5scc6[nH]ccc56)c5nsnc5c34)c2o1,0,0,0,0,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,1.70
700004,c1ncc(s1)-c1sc(-c2sc(c3Cccc23)-c2cccc3cscc23)c...,1,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.84


In [26]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
#df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

#testing purposes; comment out ID
df_test = df_test.drop(['gap'], axis=1)

In [27]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
700000,c1sc(c2[SiH2]C=Cc12)-c1cc2c3cscc3c3c4cnccc4[se...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
700001,c1cc2c3nsnc3c3cc4cc(-c5scc6[nH]ccc56)c5nsnc5c4...,0,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
700002,[nH]1c2cc(sc2c2c1c1c3ccccc3ccc1c1=CCC=c21)-c1c...,1,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
700003,c1cc2oc3c(ccc4cc(-c5scc6[nH]ccc56)c5nsnc5c34)c2o1,0,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
700004,c1ncc(s1)-c1sc(-c2sc(c3Cccc23)-c2cccc3cscc23)c...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [28]:
#convert to molecules
mols = [Chem.MolFromSmiles(m) for m in df_all.smiles.astype(str)]

In [29]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)

#counting functions for c, C, nH, =, #
smiles_nH = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('nH')))
df_all['smiles_nH'] = pd.DataFrame(smiles_nH)

smiles_e = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('=')))
df_all['smiles_e'] = pd.DataFrame(smiles_e)

smiles_n = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('#')))
df_all['smiles_n'] = pd.DataFrame(smiles_n)

In [30]:
#functions for molecules
mols_aro = np.vstack(AllChem.CalcNumAromaticRings(m) for m in mols)
df_all['mols_aro'] = pd.DataFrame(mols_aro)

In [31]:
mols_amide = np.vstack(AllChem.CalcNumAmideBonds(m) for m in mols)
df_all['mols_amide'] = pd.DataFrame(mols_amide)

In [32]:
mols_rings = np.vstack(AllChem.CalcNumRings(m) for m in mols)
df_all['mols_rings'] = pd.DataFrame(mols_rings)

In [33]:
mols_spiro = np.vstack(AllChem.CalcNumSpiroAtoms(m) for m in mols)
df_all['mols_spiro'] = pd.DataFrame(mols_spiro)

In [34]:
mols_hetero = np.vstack(AllChem.CalcNumHeteroatoms(m) for m in mols)
df_all['mols_hetero'] = pd.DataFrame(mols_hetero)

In [ ]:
mols_hbd = np.vstack(AllChem.CalcNumHBD(m) for m in mols)
df_all['mols_hbd'] = pd.DataFrame(mols_hbd)

In [ ]:
mols_hba = np.vstack(AllChem.CalcNumHBA(m) for m in mols)
df_all['mols_hba'] = pd.DataFrame(mols_hba)

In [35]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (100001, 264)
Train gap: (100001,)
Test features: (400001, 264)


In [28]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [36]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [19]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [20]:
#write_to_file("sample1.csv", LR_pred)
#write_to_file("sample2.csv", RF_pred)

write_to_file("output.csv", RF_pred)